In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver import ChromeOptions
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
from bs4 import BeautifulSoup

/tmp/ipykernel_38052/1023897703.py:8: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
service = Service()
options = ChromeOptions()


In [3]:
driver = webdriver.Chrome(service=service, options=options)



In [4]:
def pesquisa_processo(cpf = None, nome = None):
    if nome and not cpf:
        info = nome
        campo_pesquisa = driver.find_element(By.XPATH, '//*[@id="fPP:dnp:nomeParte"]')
    elif cpf and  not nome:
        info = cpf
        campo_pesquisa = driver.find_element(By.XPATH, '//*[@id="fPP:dpDec:documentoParte"]')
    
    campo_pesquisa.click()
    campo_pesquisa.send_keys(info)
    
    driver.find_element(By.XPATH, '//*[@id="fPP:searchProcessos"]').click()
    
def lista_processos():
    
    num_resultado = WebDriverWait(driver, 200).until(
            EC.visibility_of_element_located((By.XPATH, '//*[@id="fPP:processosTable:tb"]/tr[1]'))
        )
    tabela = driver.find_element(By.XPATH, '//*[@id="fPP:processosGridPanel_body"]')
    
    html_tabela = tabela.get_attribute('outerHTML')
    soup = BeautifulSoup(html_tabela, 'html.parser')
    
    dados_tabela = []

    for linha in soup.find_all('tr'):
        dados_linha = []
        
        link_tag = linha.find('a', class_= 'btn btn-default btn-sm')
        if link_tag:
            link = link_tag.get('onclick').split("'")[3]  # extrai o link do atributo onclick
            link = 'https://pje1g.trf1.jus.br' + link
            
        dados_linha += [coluna.get_text(strip=True) for coluna in linha.find_all(['th', 'td'])]
        
        if 'Ver detalhes do processo' in dados_linha:
            indice = dados_linha.index('Ver detalhes do processo')
            dados_linha[indice] = link
        
        dados_tabela.append(dados_linha)
    

    df = pd.DataFrame(dados_tabela, columns=['link', 'processo', 'ultima_movimentacao'])
    df = df.iloc[2:]
    df.reset_index(drop=True, inplace=True)
    
    return df

def save_csv(df):
    df.to_csv('lista_processos', index = False)


In [5]:
url = 'https://pje1g.trf1.jus.br/consultapublica/ConsultaPublica/listView.seam'
driver.get(url)

nome = 'Iran Velasco Nascimento'
pesquisa_processo(nome=nome)
df = lista_processos()
save_csv(df)

In [6]:
df

,link,processo,ultima_movimentacao
0,https://pje1g.trf1.jus.br/consultapublica/Cons...,CUMPRIMENTO DE SENTENÇACumSen 0039504-91.2004....,Juntada de certidão (01/08/2022 16:27:12)


In [3]:
import requests

def obter_html(url):
    try:
        # Envia uma solicitação GET para a URL especificada
        response = requests.get(url)
        
        # Verifica se a solicitação foi bem-sucedida (código de status 200)
        if response.status_code == 200:
            # Retorna o conteúdo HTML da página
            return response.text
        else:
            print(f"Erro: Status Code {response.status_code}")
            return None
    except requests.RequestException as e:
        print(f"Erro ao solicitar a página: {e}")
        return None

if __name__ == "__main__":
    url = "https://8de1-2804-3538-505-ca66-d89-eec1-81f-1897.ngrok-free.app"
    html = obter_html(url)
    if html:
        print(html)


<p>Ola, mundo!</p>


In [4]:
str_original = 'ultima_movimentacao": "Juntada de certid\u00e3o (01/08/2022 16:27:12)'

bytes_encoded = str_original.encode(encoding='utf-8')
#print(type(bytes_encoded))

#str_decoded = bytes_encoded.decode()
#print(type(str_decoded))

print(str_original.decode())

#print('Encoded bytes =', bytes_encoded)
#print('Decoded String =', str_decoded)
#print('str_original equals str_decoded =', str_original == str_decoded)

<class 'bytes'>
<class 'str'>
Encoded bytes = b'ultima_movimentacao": "Juntada de certid\xc3\xa3o (01/08/2022 16:27:12)'
Decoded String = ultima_movimentacao": "Juntada de certidão (01/08/2022 16:27:12)
